# Top des recherches les plus faites par mois

**Définition** : possible bug sur le fait que la requête "procédure de licenciement salarié à domicile" soit apparau en tête des recherches sur le site depuis quelques mois alors qu'on ne l'avait jamais vu. L'objectif de ce notebook est donc de reconstruire les recherches les plus faites

De même pour la recherche 3239

**Formule** : 

## On commence par le chargement des données
on décommente ce dont on a besoin

In [ ]:
import pandas as pd
import time
import json
import math
from tqdm.notebook import tqdm
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
#from typing import List, Dict


pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 250)
pd.set_option('display.max_colwidth', 150)

## On récupère les données via celle déjà télécharger

In [ ]:
from join_multiple_file import pd_read_pattern, PATH_CDTN_MONOLOG

logs_july = pd_read_pattern(PATH_CDTN_MONOLOG + "/data-all-logs-july/*")

In [ ]:
logs_july.shape

In [ ]:
logs_july.head()

In [ ]:
def format_url_and_get_month_and_datetime_columns(df: pd.DataFrame):
    df['url'] = df['url'].str.split('#').str[0].str.split('?').str[0]
    df['month'] = pd.DatetimeIndex(df['logfile']).month
    df['datetime'] = df['timestamp'].apply(datetime.fromtimestamp)
    return df

In [ ]:
def clean_dataset_from_useless_rows(df: pd.DataFrame):
    return df[df['type'].isin(['visit_content', 'outil', 'themes', 'select_result', 'search', 'home',
                                    'pagecc_clickcontrib', 'modeles_courriers'])]

In [ ]:
logs = pd.concat([logs_july]).copy()

In [ ]:
logs = format_url_and_get_month_and_datetime_columns(logs)

In [ ]:
#cleaned_logs = clean_dataset_from_useless_rows(logs.copy())

# Début de l'explo

On va commencer par récupérer les données des recherches et on va préprocesser un peu les recherches

Commencons par la recherche **procédure de licenciement salarié à domicile**

### Préparation des données

In [ ]:
logs['query'] = logs['query'].str.lower()

In [ ]:
searches = logs[logs['type'] == 'search'].copy()

In [ ]:
searches.groupby('query').agg({'idVisit':'nunique'}).sort_values('idVisit', ascending=False).head(20)

Effectivement on trouve un résultat très proche que ce que nous propose matomo :
https://matomo.fabrique.social.gouv.fr/index.php?module=CoreHome&action=index&idSite=4&period=day&date=yesterday&updated=1#?idSite=4&period=month&date=2022-07-01&segment=&category=General_Actions&subcategory=Actions_SubmenuSitesearch

On va aller regarder qui sont ses visiteurs qui font ces query

In [ ]:
idVisiteursWithQueryLicenciementSalarieDomicile = searches[
    searches['query']=='procédure de licenciement salarié à domicile'
]['idVisit'].unique().tolist()

In [ ]:
searches[
    searches['query']=='procédure de licenciement salarié à domicile'
].groupby('referrerName').agg({'idVisit': 'nunique'}).sort_values('idVisit', ascending=False).head(20)

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

for idVisit in idVisiteursWithQueryLicenciementSalarieDomicile[:100]:
    display(logs[logs['idVisit'] == idVisit][
        ['timeSpent', 'datetime', 'url', 'idVisit', 'referrerName', 'type', 'query', 'outil', 'outilAction', 'resultSelection', 'suggestionPrefix', 'suggestionSelection', 'recoSelection']
    ])
    

### De même avec la requête 3239

In [ ]:
searches[
    searches['query']=='3239'
].groupby('referrerName').agg({'idVisit': 'nunique'}).sort_values('idVisit', ascending=False).head(20)

## Généralisation

Peut être serait-il intéressant de faire ça pour le top 10 des recherches

In [ ]:
topQueries = searches.groupby('query').agg({'idVisit':'nunique'}).sort_values('idVisit', ascending=False).head(30).index.tolist()

In [ ]:
for query in topQueries:
    print(query)
    display(
        searches[searches['query']==query]\
        .groupby('referrerName')\
        .agg({'idVisit': 'nunique'})\
        .sort_values('idVisit', ascending=False)\
        .head(5)
    )
    print('_______________________________________________________________________________________\n\n')